In [1]:
import pandas as pd
import numpy as np
from scipy.stats import zscore, mstats

# Let's try to replicate the MSCI Momentum Index methodology

- Uses both 6-month and 12-month returns
- Final score winsorized at +/-3
- Final weight capped at 7%

- Exclude some tickers that would duplicate A vs B shares:

In [2]:
# Define a list of tickers to exclude
tickers_to_exclude = ['GOOG-USA', 'NWS-USA', 'FOX-USA']

In [3]:
# Load market data
market_data = pd.read_csv('market_data.csv')

# Convert date column to datetime
market_data['date'] = pd.to_datetime(market_data['date'])

# Filter out the specified tickers
market_data = market_data[~market_data['Ticker'].isin(tickers_to_exclude)]

# Extract year and month from date
market_data['year_month'] = market_data['date'].dt.to_period('M')

# Sort data by Ticker and date
market_data = market_data.sort_values(by=['Ticker', 'date'])

# Calculate monthly closing prices
monthly_prices = market_data.groupby(['Ticker', 'year_month']).agg({'Price': 'last'}).reset_index()

# Create a pivot table with Tickers as rows and year_month as columns
pivot_prices = monthly_prices.pivot(index='Ticker', columns='year_month', values='Price')

In [4]:
pivot_prices

year_month,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07
Ticker,,,,,,,,,,,,,
A-USA,121.77,121.07,111.82,103.37,127.80,139.03,130.10,137.36,145.51,137.04,130.41,129.63,139.49
AAL-USA,16.75,14.73,12.81,11.15,12.43,13.74,14.23,15.68,15.35,13.51,11.50,11.33,10.76
AAPL-USA,196.45,187.87,171.21,170.77,189.95,192.53,184.40,180.75,171.48,170.33,192.25,210.62,218.80
ABBV-USA,149.58,146.96,149.06,141.18,142.39,154.97,164.40,176.05,182.10,162.64,161.24,171.52,186.78
ABNB-USA,152.19,131.55,137.21,118.29,126.34,136.14,144.14,157.47,164.96,158.57,144.93,151.63,138.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYL-USA,112.75,103.54,91.03,93.54,105.13,114.36,112.44,127.05,129.24,130.70,141.02,135.63,133.48
YUM-USA,137.67,129.38,124.94,120.86,125.55,130.66,129.49,138.42,138.65,141.25,137.43,132.46,132.17
ZBH-USA,138.15,119.12,112.22,104.41,116.31,121.70,125.60,124.36,131.98,120.28,115.15,108.53,111.37


In [5]:
# Calculate weekly returns
market_data['weekly_return'] = market_data.groupby('Ticker')['Price'].pct_change()

# Filter the data to the last 3 years
three_years_ago = pd.Timestamp.now() - pd.DateOffset(years=3)
market_data_3y = market_data[market_data['date'] >= three_years_ago]

# Calculate annualized standard deviation of weekly returns for each stock
annualized_std = market_data_3y.groupby('Ticker')['weekly_return'].std() * (52 ** 0.5)
annualized_std = annualized_std.rename('Annualized Std')

/tmp/ipykernel_2198/399203024.py:2: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  market_data['weekly_return'] = market_data.groupby('Ticker')['Price'].pct_change()


In [6]:
# Calculate 6-month price change excluding the most recent month
momentum_value_6m = (pivot_prices.iloc[:, -2] / pivot_prices.iloc[:, -7]) - 1

# Create DataFrame for 6-month momentum
momentum_df_6m = pd.DataFrame(momentum_value_6m, columns=['Momentum Value 6M'])

# Calculate 12-month price change excluding the most recent month
momentum_value_12m = (pivot_prices.iloc[:, -2] / pivot_prices.iloc[:, -13]) - 1

# Calculate the 12-month price change excluding the most recent month (already calculated as momentum_value)
momentum_df_12m = pd.DataFrame(momentum_value_12m, columns=['Momentum Value 12M'])

# Merge 6-month and 12-month momentum DataFrames
momentum_df = momentum_df_6m.join(momentum_df_12m)

# Filter out rows where either 6M or 12M momentum is zero
momentum_df = momentum_df[(momentum_df['Momentum Value 6M'] != 0) & (momentum_df['Momentum Value 12M'] != 0)]

# Merge annualized standard deviation with the momentum DataFrame
momentum_df = momentum_df.merge(annualized_std, left_index=True, right_index=True)

# Calculate risk-adjusted momentum for both 6-month and 12-month momentum values
momentum_df['Risk-Adjusted Momentum 6M'] = momentum_df['Momentum Value 6M'] / momentum_df['Annualized Std']
momentum_df['Risk-Adjusted Momentum 12M'] = momentum_df['Momentum Value 12M'] / momentum_df['Annualized Std']

# Calculate weighted average of risk-adjusted momentum
momentum_df['Weighted Risk-Adjusted Momentum'] = (momentum_df['Risk-Adjusted Momentum 6M'] + momentum_df['Risk-Adjusted Momentum 12M']) / 2

# Merge market cap data
latest_market_data = market_data.sort_values('date').drop_duplicates('Ticker', keep='last')
momentum_df = momentum_df.merge(latest_market_data[['Ticker', 'Market Cap (millions)']], left_index=True, right_on='Ticker')

In [7]:
# Drop rows where required columns have NaN values
momentum_df.dropna(subset=['Risk-Adjusted Momentum 6M', 'Risk-Adjusted Momentum 12M', 'Market Cap (millions)'], inplace=True)

# Standardize the 'Weighted Risk-Adjusted Momentum' with z-scores
momentum_df['Final Z-score'] = zscore(momentum_df['Weighted Risk-Adjusted Momentum'])

# Winsorize the final Z-scores (limit extreme values)
momentum_df['Clipped Z-score'] = momentum_df['Final Z-score'].clip(-3, 3)

# Calculate the Momentum Score as per MSCI methodology
def compute_momentum_score(z):
    if z > 0:
        return 1 + z
    else:
        return (1 - z) ** -1

momentum_df['Momentum Score'] = momentum_df['Clipped Z-score'].apply(compute_momentum_score)

# Sort and limit to top 125 by Momentum Score
momentum_df = momentum_df.sort_values(by="Momentum Score", ascending=False).head(125)

# Remove '-USA' suffix from tickers in the momentum_df
momentum_df['Ticker'] = momentum_df['Ticker'].str.replace('-USA', '')

In [8]:
# Calculate weights based on Clipped Z-score and Market Cap
momentum_df['Weight_unnormalized'] = momentum_df['Clipped Z-score'] * momentum_df['Market Cap (millions)']
total_weight = momentum_df['Weight_unnormalized'].sum()
momentum_df['Final Weight'] = (momentum_df['Weight_unnormalized'] / total_weight) * 100

In [9]:
momentum_df = momentum_df.sort_values(by="Final Weight",ascending=False)
momentum_df

,Momentum Value 6M,Momentum Value 12M,Annualized Std,Risk-Adjusted Momentum 6M,Risk-Adjusted Momentum 12M,Weighted Risk-Adjusted Momentum,Ticker,Market Cap (millions),Final Z-score,Clipped Z-score,Momentum Score,Weight_unnormalized,Final Weight
91175,1.007899,1.643754,0.210718,4.783173,7.800745,6.291959,NVDA,2.551758e+06,3.536372,3.000000,4.000000,7.655274e+06,23.314917
85673,0.124176,0.330525,0.087743,1.415232,3.766981,2.591106,MSFT,3.143271e+06,1.215931,1.215931,2.215931,3.822001e+06,11.640294
8645,0.245168,0.445616,0.123479,1.985499,3.608842,2.797171,AMZN,1.890988e+06,1.345134,1.345134,2.345134,2.543632e+06,7.746892
74931,0.402364,0.991816,0.137548,2.925266,7.210706,5.067986,LLY,7.510199e+05,2.768939,2.768939,3.768939,2.079528e+06,6.333414
55805,0.300143,0.372438,0.122673,2.446696,3.036032,2.741364,GOOGL,9.977291e+05,1.310143,1.310143,2.310143,1.307168e+06,3.981112
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73097,0.025803,0.192977,0.063893,0.403846,3.020315,1.712080,L,1.781878e+04,0.664780,0.664780,1.664780,1.184557e+04,0.036077
39299,0.281158,0.358663,0.157876,1.780876,2.271800,2.026338,DVA,1.196842e+04,0.861820,0.861820,1.861820,1.031463e+04,0.031414
64975,0.204298,0.196617,0.127332,1.604449,1.544124,1.574286,IP,1.595122e+04,0.578383,0.578383,1.578383,9.225918e+03,0.028098
42443,0.172591,0.144777,0.100495,1.717418,1.440642,1.579030,EMN,1.206817e+04,0.581358,0.581358,1.581358,7.015925e+03,0.021368


### Add caps to portfolio

MSCI caps the weight, though it's not totally clear what the methodology is to do that. They also have rules during reconstitution.
Let's cap the weight at 7%, which is around NVDA weight in SPMO, then ensure the rest of the portfolio is weighted accordingly.

In [10]:
# Sample data preparation (assuming previous calculations and DataFrame `momentum_df`)
# For illustration, making sure the column names match your data.
# Ensure that the DataFrame is loaded or created with appropriate columns
momentum_df['Weight_unnormalized'] = momentum_df['Market Cap (millions)'] * momentum_df['Momentum Score']
total_weight = momentum_df['Weight_unnormalized'].sum()
momentum_df['Final Weight'] = (momentum_df['Weight_unnormalized'] / total_weight) * 100  # Normalize to percentages

# Cap weights at 7%
momentum_df['Capped Weight'] = momentum_df['Final Weight'].clip(upper=7)

# Calculate the excess weight from capping
excess_weight = momentum_df['Final Weight'].sum() - momentum_df['Capped Weight'].sum()

# Handle cases where weights should be redistributed
iteration = 0
while excess_weight > 0.01 and iteration < 10:  # Using tolerance of 0.01 and max iterations to prevent infinite loops
    iteration += 1
    # Compute weights of securities below 7%
    not_capped = momentum_df['Capped Weight'] < 7.0
    
    # Total weight of securities that are not capped but still need redistribution
    not_capped_total_weight = momentum_df.loc[not_capped, 'Capped Weight'].sum()
    
    # If no weights are to be redistributed, break the loop
    if not_capped_total_weight == 0:
        break
    
    # Calculate the amount to redistribute
    redistribution = excess_weight / not_capped_total_weight
    
    # Apply redistribution, ensuring no weight exceeds the cap
    momentum_df.loc[not_capped, 'Capped Weight'] += (momentum_df.loc[not_capped, 'Capped Weight'] * redistribution).clip(upper=(7 - momentum_df.loc[not_capped, 'Capped Weight']))

    # Recalculate the excess weight
    excess_weight = 100 - momentum_df['Capped Weight'].sum()

# Assign the adjusted final weights
momentum_df['Adjusted Final Weight'] = momentum_df['Capped Weight']

# Remove '-USA' suffix from tickers in the momentum_df
momentum_df['Ticker'] = momentum_df['Ticker'].str.replace('-USA', '')

In [11]:
momentum_df = momentum_df.round(decimals=2)
momentum_df

,Momentum Value 6M,Momentum Value 12M,Annualized Std,Risk-Adjusted Momentum 6M,Risk-Adjusted Momentum 12M,Weighted Risk-Adjusted Momentum,Ticker,Market Cap (millions),Final Z-score,Clipped Z-score,Momentum Score,Weight_unnormalized,Final Weight,Capped Weight,Adjusted Final Weight
91175,1.01,1.64,0.21,4.78,7.80,6.29,NVDA,2551758.00,3.54,3.00,4.00,10207032.00,18.86,7.00,7.00
85673,0.12,0.33,0.09,1.42,3.77,2.59,MSFT,3143270.73,1.22,1.22,2.22,6965271.86,12.87,7.00,7.00
8645,0.25,0.45,0.12,1.99,3.61,2.80,AMZN,1890988.19,1.35,1.35,2.35,4434620.54,8.20,7.00,7.00
74931,0.40,0.99,0.14,2.93,7.21,5.07,LLY,751019.85,2.77,2.77,3.77,2830547.82,5.23,6.88,6.88
55805,0.30,0.37,0.12,2.45,3.04,2.74,GOOGL,997729.11,1.31,1.31,2.31,2304896.79,4.26,5.60,5.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73097,0.03,0.19,0.06,0.40,3.02,1.71,L,17818.78,0.66,0.66,1.66,29664.35,0.05,0.07,0.07
39299,0.28,0.36,0.16,1.78,2.27,2.03,DVA,11968.42,0.86,0.86,1.86,22283.05,0.04,0.05,0.05
64975,0.20,0.20,0.13,1.60,1.54,1.57,IP,15951.22,0.58,0.58,1.58,25177.14,0.05,0.06,0.06
42443,0.17,0.14,0.10,1.72,1.44,1.58,EMN,12068.17,0.58,0.58,1.58,19084.10,0.04,0.05,0.05


In [12]:
momentum_df.to_csv("momentum_MSCI_proxy.csv")

### How closely does this match the MSCI portfolio?

In [13]:
# Load CSV data
momentum_proxy_df = pd.read_csv("momentum_MSCI_proxy.csv")
mtum_holdings_df = pd.read_csv("MTUM_holdings.csv")

# Strip trailing spaces from the 'Ticker' column in mtum_holdings_df
mtum_holdings_df['Ticker'] = mtum_holdings_df['Ticker'].str.strip()

# Part 1: Finding common tickers
momentum_tickers = set(momentum_proxy_df['Ticker'])
mtum_tickers = set(mtum_holdings_df['Ticker'])

common_tickers = momentum_tickers.intersection(mtum_tickers)
num_common_tickers = len(common_tickers)

print(f"Number of common tickers: {num_common_tickers}")
print(f"Common tickers: {common_tickers}")

Number of common tickers: 64
Common tickers: {'GD', 'CTAS', 'TT', 'MU', 'UBER', 'GE', 'MCK', 'STX', 'WM', 'LDOS', 'VST', 'AXP', 'GM', 'ALL', 'BSX', 'LLY', 'SMCI', 'AXON', 'RCL', 'AMAT', 'NVDA', 'WAB', 'LRCX', 'FICO', 'DECK', 'C', 'GDDY', 'APH', 'AVGO', 'NTAP', 'AMZN', 'PGR', 'DVA', 'WFC', 'RSG', 'TDG', 'CMG', 'SYF', 'NFLX', 'NRG', 'FI', 'DPZ', 'PWR', 'GRMN', 'META', 'HIG', 'ANET', 'JPM', 'NXPI', 'CEG', 'COST', 'ISRG', 'CRWD', 'HLT', 'ETN', 'IR', 'QCOM', 'CAT', 'CPRT', 'WDC', 'KKR', 'L', 'KLAC', 'HWM'}


In [14]:
# Extract and rename columns
momentum_proxy_df = momentum_proxy_df[['Ticker', 'Adjusted Final Weight']]
momentum_proxy_df = momentum_proxy_df.rename(columns={'Adjusted Final Weight': 'Proxy Weight'})

mtum_holdings_df = mtum_holdings_df[['Ticker', 'Weight (%)']]
mtum_holdings_df = mtum_holdings_df.rename(columns={'Weight (%)': 'MTUM Weight'})

# Merge the dataframes on 'Ticker', using outer join to include all tickers
combined_df = pd.merge(momentum_proxy_df, mtum_holdings_df, on='Ticker', how='outer')

# Fill NaN values with 0 for weights
combined_df['Proxy Weight'].fillna(0, inplace=True)
combined_df['MTUM Weight'].fillna(0, inplace=True)

/tmp/ipykernel_2198/2991764532.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['Proxy Weight'].fillna(0, inplace=True)
/tmp/ipykernel_2198/2991764532.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

In [15]:
combined_df.sort_values(by="Proxy Weight", ascending=False).head(20)

,Ticker,Proxy Weight,MTUM Weight
11,AMZN,7.00,4.52
115,MSFT,7.00,0.00
123,NVDA,7.00,6.73
106,LLY,6.88,5.28
110,META,5.61,4.31
79,GOOGL,5.60,0.00
17,AVGO,4.26,6.30
185,WMT,3.59,0.00
98,JPM,3.53,4.72
40,COST,2.93,3.14
